In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
chunk = 100000
datafile = 'practiceMonth.csv'
tp = pd.read_csv(datafile, chunksize=chunk,
                 usecols=['Probe', 'Latitude', 'Longitude',
                          'Month', 'Depth', 'Temperature'],
                dtype={'Probe': 'str', 'Latitude': 'float32', 'Longitude': 'float32'})
prac_data = pd.concat(tp, ignore_index=True)
prac_data.dropna(inplace=True)
prac_data.head()

,Probe,Latitude,Longitude,Month,Depth,Temperature
0,animal mounted,53.633301,-161.550003,1,4.0,4.21
1,animal mounted,53.633301,-161.550003,1,10.0,4.22
2,animal mounted,53.633301,-161.550003,1,20.0,4.21
3,animal mounted,53.633301,-161.550003,1,30.0,4.20
4,animal mounted,53.633301,-161.550003,1,50.0,4.21


In [3]:
# prac_data['Probe'].unique()

In [4]:
# prac_data.shape

In [5]:
num_bins = 100
minDep = 0
maxDep = 100000
minLong = -1000  
maxLong = 1000
minLat = -1000
maxLat = 1000
depthchunk = 10
areachunk = 1
#list of probes ['animal mounted', 'CTD', 'XCTD', 'drifting buoy', 'glider', 'moored buoy', 'bottle/rosette/net', 'profiling float', 'towed CTD', 'XBT']
probes = ['animal mounted', 'CTD', 'XCTD', 'drifting buoy', 'glider', 'moored buoy', 'bottle/rosette/net', 'profiling float', 'towed CTD']

In [6]:
# prac_data[prac_data['Probe'].isin(probes)].shape
# prac_data.loc[(minDep < prac_data['Depth']) & (prac_data['Depth'] < maxDep)].shape
# prac_data.loc[(minLat < prac_data['Latitude']) & (prac_data['Latitude'] < maxLat)].shape
# prac_data.loc[(minLong < prac_data['Longitude']) & (prac_data['Longitude'] < maxLong)].shape

In [7]:
prac_data = prac_data[prac_data['Probe'].isin(probes)]
prac_data = prac_data.loc[(minDep < prac_data['Depth']) & (prac_data['Depth'] < maxDep)]
prac_data = prac_data.loc[(minLat < prac_data['Latitude']) & (prac_data['Latitude'] < maxLat)]
prac_data = prac_data.loc[(minLong < prac_data['Longitude']) & (prac_data['Longitude'] < maxLong)]
prac_data.head()

prac_data.shape

(4282957, 6)

In [8]:
# prac_data['Probe'].unique()

In [10]:
# histogram generation test
temp_data = prac_data
# temp_data = temp_data[temp_data['Probe'] == 'bottle/rosette/net']
temp_data = temp_data[temp_data['Depth'].between(0, 1000000000)]
temp_data = temp_data[temp_data['Longitude'].between(minLong, maxLong)]
temp_data = temp_data[temp_data['Latitude'].between(minLat, maxLat)]
print(temp_data.head())
temp_data.to_csv(
    f'Data.csv')
# x is depth, y is temperature
# print(prac_data)
y = temp_data["Temperature"]
# sigmas.append(np.std(temp_data["Temperature"]))
#print (y)
plt.hist(y, 1000)
plt.xlabel('Temperature (C)')
plt.ylabel('Frequency')
plt.title(f'Temperature Histogram for All Locations (practiceMonth)\n', fontweight="bold")
plt.savefig(
    f'Histogram.png')
plt.clf()


            Probe   Latitude   Longitude  Month  Depth  Temperature
0  animal mounted  53.633301 -161.550003      1    4.0         4.21
1  animal mounted  53.633301 -161.550003      1   10.0         4.22
2  animal mounted  53.633301 -161.550003      1   20.0         4.21
3  animal mounted  53.633301 -161.550003      1   30.0         4.20
4  animal mounted  53.633301 -161.550003      1   50.0         4.21


<Figure size 640x480 with 0 Axes>

In [14]:
# max(temp_data['Depth'])

In [ ]:
sigmas = []
temps = []
for lat in range(minLat, maxLat, areachunk):
    for long in range(minLong, maxLong, areachunk):
        for dep in range(0, maxDep, depthchunk):

            temp_data = temp_data[temp_data['Depth'].between(dep, dep+depthchunk)]
            temp_data = temp_data[temp_data['Latitude'].between(lat, lat+areachunk)]
            temp_data = temp_data[temp_data['Longitude'].between(long, long+areachunk)]
            temp_data.to_csv(
                f'Data_long_{long}_{long+areachunk}_lat_{lat}_{lat+areachunk}_dep_{dep}_{dep+depthchunk}.csv')

            # x is depth, y is temperature
            y = temp_data["Temperature"]
            plt.hist(y, 100)
            plt.xlabel('Temperature (C)')
            plt.ylabel('Frequency')
            plt.title(f'Long: {long}-{long+areachunk}, Lat: {lat}-{lat+areachunk}, Dep: {dep}-{dep+depthchunk} Histogram\n',
                      fontweight="bold")
            plt.savefig(
                f'Histogram_long_{long}_{long+areachunk}_lat_{lat}_{lat+areachunk}_dep_{dep}_{dep+depthchunk}.png')
            plt.clf()
            
            sigmas.append(np.std(temp_data["Temperature"]))
            temps += y
        # Creating plot
        plt.boxplot(temps)
        plt.xlabel('Temperature (C)')
        plt.ylabel('Frequency')
        plt.title(f'Temperature Boxplot for Lat:{lat}-{lat+areachunk} Long:{long}-{long+areachunk}')
        plt.savefig(
            f'Boxplot.png')
        plt.clf()
        
        # print(sigmas)
        plt.plot(sigmas)
        plt.xlabel('Depth Range')
        plt.ylabel('Standard Deviation')
        plt.title(f'Temperature STD Curve for Lat:{lat}-{lat+areachunk} Long:{long}-{long+areachunk}')
        plt.savefig(
            f'STDCurve.png')
        
        sigmas = []
        temps = []



In [ ]:
# inputs = lat, long, month, depth
# use builtin pandas functions to filter data and get std
# compile data into a csv based on inputs
# use square area to get latitue and longitude range

